# Example 0.2: Compressor Isentropic Efficiency
*John F. Maddox, Ph.D., P.E.<br>
University of Kentucky - Paducah Campus<br>
ME 321: Engineering Thermodynamics II<br>*

## Problem Statement
An air compressor has an isentropic efficiency of 80% and operates in a steady-state, steady-flow (SSSF) configuration with negligible changes in kinetic and potential energy.  It receives a volumetric flow rate of 3000 CFM with an inlet pressure of $p_1=14.7\,\text{psia}$ and inlet temperature of $T_1=70^\circ\text{F}$.  It compresses the air by a factor of 10. Determine<br>
(a) Rate of compressor work, $\mathrm{HP}$<br>
(b) Rate of entropy generation, $\mathrm{Btu/min^\circ R}$

## Solution
In the previous example (Ex 0.1), we wrote two separate python scripts to illustrate the difference between using only the standard python library and using third-party modules.  Those two scripts were placed in two self-contained code blocks to help show the separation between the two.  In this example we will jump straight to using the third-party libraries to make things easier, and we will spread the python code out across multiple cells with explanatory text (markdown) cells to describe the code rather than using python comments.  To execute this code you will need to execute each code block in order, or select the `Run All` option from the `Cell` menu above.

### Python Initialization
We'll start by importing the libraries we will use for our analysis.  

In [1]:
from math import log
from kilojoule.templates.USCS_R import *

/opt/anaconda/lib/python3.8/site-packages/mpmath/ctx_mp_python.py:892: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if other is 0:
/opt/anaconda/lib/python3.8/site-packages/mpmath/ctx_mp_python.py:986: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if other is 0:


In the first version of the previous example we defined a new variable for each property at each state, but in this example (and future examples) we will instead store the values in a custom python data structure from `kilojoule`.  This is a different approach to variable naming and organization that will allow us to do some interesting things later on.  By import the `USCS_F` template from `kilojoule` above, we have defined a few common variables and settings that are useful for thermodynamic calculations when using English units with temperature in $^\circ\mathrm{F}$ (*Note: templates are available for other unit systems or you can develop a custom template if you prefer*).  In particular, the `states` variable been has configured to hold the properties at our various states.

We can also use the `kilojoule` library to look up property values for real fluid using the `realfluid.Properties()` class or for ideal gasses using the `idealgas.Properties()`.  For each of these functions you initialize (instantiate) the class by passing it the name of the fluid and the an optional preferred unit system to use when returning values (default: SI in $\mathrm{^\circ C}$.  For this case, we will treat the air as an ideal gas, so we will use the `idealgas.Properties()` class.

In [2]:
air = idealgas.Properties('Air')

### Given Parameters
We now define variables to hold our known values.

In [3]:
# the next three lines show different ways of achieving the result of creating a dimensional quantity, but the Q_() syntax is the preferred method.
T[1] = Quantity(70.,'degF')              # inlet temperature
p[1] = 14.7*units.psi              # inlet pressure
Vdot[1] = 3000.0*units('ft^3/min') # volumetric flow rate at inlet
p[2] = 10*p[1]                     # exit pressure
eta_c = Quantity(0.8,'')                 # isentropic efficiency

display.Summary(locals());

<IPython.core.display.Latex object>

,T,p,Vdot
unit,°R,psi,ft³/s
1,529.67,14.7,50
2,-,147.0,-



### Assumptions
  - Negligible changes in kinetic energy 
  - Negligible changes in potential energy 
  - Adiabatic (no heat transfer)
  - Constant specific heat (cold-air-standard)
  - Ideal gas (cold-air-standard)

We could pull properties for air from the tables in the back of the book since we are assuming constant specific heat and ideal gas behavior, or we can look them using `air` reference we created earlier


In [4]:
# Ideal Gas
R = air.R   # specific gas constant

# Constant thermal properties at room temperature
T_room = Quantity(25,'degC') # room temperature
c_p = air.Cp(T_room)   # constant pressure specific heat at room temperature
k = air.k(T_room)      # specific heat ratio at room temperature

display.Calculations(locals(),comments=True);

 Ideal Gas

 Constant thermal properties at room temperature

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

### Isentropic Efficiency
The isentropic efficiency of a compressor is defined as the ratio of the work that would be required if the compressor were ideal (isentropic) and operating between the same inlet state and exit pressure as the real device to the actual work.
$$\eta_c=\frac{\dot{W}_s}{\dot{W}_c}$$
where $\dot{W}_s$ is the rate of isentropic work and $\dot{W}_c$ is the rate of actual compressor work.  From a first law analysis, we can rewrite the work terms as changes in enthalpy between the inlet and exit states.
$$\require{cancel}
\eta_c = \frac{\cancel{\dot{m}}(h_{2s}-h_1)}{\cancel{\dot{m}}(h_2-h_1)}$$
Applying the constant specific heat assumption allows us to rewrite the changes in enthalpy as $\Delta h=c_p\Delta T$
$$\require{cancel}
\eta_c = \frac{\cancel{c_p}(T_{2s}-T_1)}{\cancel{c_p}(T_2-T_1)}$$
Our first goal is to find the exit temperature, so we solve for $T_2$
$$T_2 = T_1 + \frac{T_{2s}-T_1}{\eta_c}$$
However, in order to use this equation, we first need to find the temperature of the isentropic exit state, $T_{2s}$.  We can find this using ideal gas polytropic relations with $n=k$
$$T_{2s}=T_1\left(\frac{p_2}{p_1}\right)^{\frac{k-1}{k}}$$
Note that in order to apply the polytropic relation above, we must convert the temperatures to absolute units, i.e. $^\circ\text{R}$

In [5]:
T['2s'] = T[1].to('degR')*(p[2]/p[1])**((k-1)/k)
T[2] = T[1] + (T['2s'].to('degF')-T[1])/eta_c

display.Calculations(locals(),comments=True);

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

Now that we know the actual exit temperature, we can find the actual rate of work using the 1st Law.
$$\dot{W}_c = \dot{m}c_p(T_2-T_1)$$
However, we will also need to use the ideal gas law to find the mass flow rate before applying this equation.
$$\dot{m}_1 = \frac{p_1\dot{V}_1}{RT_1}$$

In [6]:
mdot = (p[1]*Vdot[1])/(R*T[1].to('degR'))
mdot = mdot.to('lb/min') # hide

Wdot_c = mdot*c_p*(T[2]-T[1])
Wdot_c = Wdot_c.to('hp') # hide

display.Calculations(locals(),comments=True);

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

### Second Law Analysis
To determine the entropy generation, we need to do a 2nd Law analysis

$$\require{cancel}
\cancelto{0}{\frac{dS_{CV}}{dt}}= \sum_j\frac{\cancelto{0}{\dot{Q}_j}}{T_j}+\sum_i\dot{m}_is_i-\sum_e\dot{m}_es_e+\dot{S}_{gen}$$
$$\dot{S}_{gen} = \dot{m}(s_e-s_i)$$
which can be rewritten using the constant specific heat assumption as
$$\dot{S}_{gen} = \dot{m}\left[ c_p\ln\left(\frac{T_2}{T_1}\right)-R\ln\left(\frac{p_2}{p_1}\right)\right]$$
where the temperatures and pressure must be in absolute units for this equation to be valid.


In [7]:
# Note: we use the `log` function from the math library for $\ln()$
Sdot_gen = mdot*( c_p*log(T[2]/T[1]) - R*log(p[2]/p[1]))

display.Calculations(locals(),comments=True);

 Note: we use the `log` function from the math library for $\ln()$

<IPython.core.display.Latex object>

Note that we used `log` from the `math` library (imported in the first cell of this notebook) to evaluate the natural log in the above equation.  It is common in many programming languages and higher level textbooks to treat the natural log, $\ln()$, as the default $\log()$ and the base 10 log is only applied if it is explicitly stated, i.e. $\log_{10}()$.

In [8]:
from math import e
log(e)

1.0

In [9]:
from math import log10
log10(10)

1.0